# Pendulum Period vs. Initial Angle

The initial boilerplate to start a sicmutils investigation:

In [50]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[org.clojure/data.json "0.2.6"]
                                [net.littleredcomputer/sicmutils "0.11.0-SNAPSHOT"]
                                [thi.ng/geom "0.0.908"]])
(ns double-pendulum
    (:refer-clojure :exclude [partial zero? + - * / ref])
    (:require [sicmutils.env :refer :all]
              [clojupyter.misc.display :as display]
              [thi.ng.geom.viz.core :as viz]
              [thi.ng.geom.svg.core :as svg]
              [thi.ng.color.core :as color]))

We choose theta and thetadot as our state variables. Define T, the kinetic energy function for the simple pendulum.

In [7]:
(defn T [m l g]
    (fn [[t theta thetadot]]
        (square (* l thetadot))))

#'double-pendulum/T

In [21]:
(defn V [m l g]
    (fn [[t theta thetadot]]
        (* -1 m g l (cos theta))))
(def L (- T V))
(defn state-derivative [m l g]
    (Lagrangian->state-derivative (L m l g)))
(tex$$ ((state-derivative 'm 'l 'g) (up 't 'theta 'thetadot)))

In [22]:
(tex$ ((L 'm 'l 'g) 
       (up 't 'theta 'thetadot))) 

In [26]:
(defn p [theta0]
    (let [out (atom [])
          observe (fn [t [_ theta thetadot]]
                      (swap! out conj [t theta]))]
        ((evolve state-derivative 1 1 9.8)
         (up 0 theta0 0)
         observe 
         1/60
         10
         1e-6
         :compile true            
        ) 
        @out))

#'double-pendulum/p

In [61]:
(def spec
  {:x-axis (viz/linear-axis
            {:domain [0 10]
             :range  [50 (- 985 10)]
             :major 1
             :pos    550})
   :y-axis (viz/linear-axis
            {:domain      [(- Math/PI) Math/PI]
             :range       [550 20]
             :major       1
             :minor       0.2
             :pos         50
             :label-dist  15
             :label-style {:text-anchor "end"}})
   :grid   {:attribs {:stroke "#caa"}
            :minor-x true
            :minor-y false}
   :data   (for [a (range 0.15 2 0.15)]
               {:values  (p a)
                :attribs {:stroke (color/as-css (color/rgba 0.8 (/ a 2) 0 1)) :stroke-width "2pt"}
                :layout  viz/svg-line-plot})
})

#'double-pendulum/spec

In [62]:
(defn export-viz
  [spec]
  (->> spec
       (viz/svg-plot2d-cartesian)
       (svg/svg {:width 985 :height 600})
       (svg/serialize)))

(-> spec
    export-viz
    display/make-html)


<?xml version="1.0"?>
 <polyline fill="none" points="50.00,272.35 51.54,272.36 53.08,272.38 54.63,272.42 56.17,272.48 57.71,272.56 59.25,272.65 60.79,272.77 62.33,272.89 63.88,273.04 65.42,273.20 66.96,273.37 68.50,273.56 70.04,273.77 71.58,273.99 73.13,274.23 74.67,274.48 76.21,274.75 77.75,275.03 79.29,275.32 80.83,275.63 82.38,275.95 83.92,276.28 85.46,276.62 87.00,276.98 88.54,277.34 90.08,277.72 91.63,278.11 93.17,278.50 94.71,278.91 96.25,279.32 97.79,279.74 99.33,280.17 100.88,280.60 102.42,281.04 103.96,281.49 105.50,281.94 107.04,282.39 108.58,282.85 110.13,283.31 111.67,283.77 113.21,284.24 114.75,284.70 116.29,285.17 117.83,285.64 119.38,286.10 120.92,286.56 122.46,287.03 124.00,287.48 125.54,287.94 127.08,288.39 128.63,288.84 130.17,289.28 131.71,289.72 133.25,290.15 134.79,290.57 136.33,290.98 137.88,291.39 139.42,291.79 140.96,292.18 142.50,292.56 144.04,292.92 145.58,293.28 147.13,293.63 148.67,293.96 150.21,294.29 151.75,294.60 153.29,294.89 154.83,295.18 156.38,295.45 157.92,295.70 159.46,295.94 161.00,296.17 162.54,296.38 164.08,296.58 165.63,296.76 167.17,296.92 168.71,297.07 170.25,297.20 171.79,297.32 173.33,297.41 174.88,297.50 176.42,297.56 177.96,297.61 179.50,297.64 181.04,297.65 182.58,297.65 184.13,297.63 185.67,297.59 187.21,297.53 188.75,297.46 190.29,297.37 191.83,297.27 193.38,297.14 194.92,297.01 196.46,296.85 198.00,296.68 199.54,296.49 201.08,296.29 202.63,296.07 204.17,295.84 205.71,295.59 207.25,295.33 208.79,295.05 210.33,294.76 211.88,294.46 213.42,294.14 214.96,293.81 216.50,293.47 218.04,293.12 219.58,292.76 221.13,292.38 222.67,292.00 224.21,291.61 225.75,291.21 227.29,290.80 228.83,290.38 230.38,289.95 231.92,289.52 233.46,289.08 235.00,288.64 236.54,288.19 238.08,287.73 239.63,287.28 241.17,286.82 242.71,286.35 244.25,285.89 245.79,285.42 247.33,284.96 248.88,284.49 250.42,284.03 251.96,283.56 253.50,283.10 255.04,282.64 256.58,282.18 258.13,281.73 259.67,281.28 261.21,280.84 262.75,280.40 264.29,279.97 265.83,279.55 267.38,279.13 268.92,278.72 270.46,278.32 272.00,277.93 273.54,277.55 275.08,277.18 276.63,276.81 278.17,276.46 279.71,276.13 281.25,275.80 282.79,275.49 284.33,275.19 285.88,274.90 287.42,274.62 288.96,274.37 290.50,274.12 292.04,273.89 293.58,273.67 295.13,273.47 296.67,273.29 298.21,273.12 299.75,272.97 301.29,272.83 302.83,272.71 304.38,272.61 305.92,272.52 307.46,272.45 309.00,272.40 310.54,272.37 312.08,272.35 313.63,272.35 315.17,272.37 316.71,272.40 318.25,272.45 319.79,272.52 321.33,272.60 322.88,272.70 324.42,272.82 325.96,272.96 327.50,273.11 329.04,273.27 330.58,273.46 332.13,273.65 333.67,273.87 335.21,274.10 336.75,274.34 338.29,274.60 339.83,274.87 341.38,275.16 342.92,275.46 344.46,275.77 346.00,276.10 347.54,276.43 349.08,276.78 350.63,277.14 352.17,277.51 353.71,277.89 355.25,278.28 356.79,278.68 358.33,279.09 359.88,279.51 361.42,279.93 362.96,280.36 364.50,280.80 366.04,281.24 367.58,281.69 369.13,282.14 370.67,282.60 372.21,283.06 373.75,283.52 375.29,283.98 376.83,284.45 378.38,284.91 379.92,285.38 381.46,285.85 383.00,286.31 384.54,286.77 386.08,287.23 387.63,287.69 389.17,288.15 390.71,288.59 392.25,289.04 393.79,289.48 395.33,289.91 396.88,290.34 398.42,290.76 399.96,291.17 401.50,291.57 403.04,291.96 404.58,292.35 406.13,292.72 407.67,293.09 409.21,293.44 410.75,293.78 412.29,294.11 413.83,294.43 415.38,294.73 416.92,295.02 418.46,295.30 420.00,295.57 421.54,295.81 423.08,296.05 424.63,296.27 426.17,296.47 427.71,296.66 429.25,296.83 430.79,296.99 432.33,297.13 433.88,297.26 435.42,297.36 436.96,297.45 438.50,297.53 440.04,297.58 441.58,297.62 443.13,297.65 444.67,297.65 446.21,297.64 447.75,297.61 449.29,297.57 450.83,297.50 452.38,297.42 453.92,297.33 455.46,297.21 457.00,297.08 458.54,296.94 460.08,296.77 461.63,296.60 463.17,296.40 464.71,296.19 466.25,295.97 467.79,295.73 469.33,295.47 470.88,295.20 472.42,294.92 473.96,294.63 475.50,294.32 477.04,293.99 478.58,293.66 480.13,293.32 481.67,292.96 483.21,292.59 484.75,292.21 486.29,291.82 487.